# MEDIAR Experiments

The framework for most of the experiments is conducted by loading the corresponding model from google drive using the ```gdown``` command. Then, a configuration file is either created or modified from previous experiments (```config/``` directory). Our experimental process/reasoning is more clearly explained in the project report.



---


## 1. Setup

Clone Github repository and install dependencies.

In [ ]:
# Clone MEDIAR github repo
!git clone https://github.com/CS-433/ml-project-2-doughminators.git
%rm -rf ./sample_data
%cd ml-project-2-doughminators/

# Install dependencies
%pip install -r requirements.txt
%pip install segmentation-models-pytorch==0.3.1
%pip install wandb


!wandb offline

# Uncomment and add login key to record experiments
# !wandb online
# !wandb login [ insert wandb key ]

#### Download MEDIAR Models

In [2]:
# Download MEDIAR pretrained weights
%mkdir weights
!gdown https://drive.google.com/uc?id=168MtudjTMLoq9YGTyoD2Rjl_d3Gy6c_L -O weights/from_phase1.pth
!gdown https://drive.google.com/uc?id=1JJ2-QKTCk-G7sp5ddkqcifMxgnyOrXjx -O weights/from_phase2.pth

Downloading...
From: https://drive.google.com/uc?id=168MtudjTMLoq9YGTyoD2Rjl_d3Gy6c_L
To: /content/ml-project-2-doughminators/weights/from_phase1.pth
100% 486M/486M [00:06<00:00, 70.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1JJ2-QKTCk-G7sp5ddkqcifMxgnyOrXjx
To: /content/ml-project-2-doughminators/weights/from_phase2.pth
100% 486M/486M [00:10<00:00, 46.0MB/s]


#### Load packages

In [3]:
import torch
import glob
import skimage.io as io
import matplotlib.pyplot as plt
import numpy as np

from train_tools import *
from train_tools.models import MEDIARFormer
from core.MEDIAR import Predictor, EnsemblePredictor

If one wants to use the A100 NVIDIA GPUs on colab, a new versino of torch needs to be installed. For CUDA compatibility when using A100, run this cell and restart kernel.

In [ ]:
!pip install torch==1.11.0+cu113 --extra-index-url https://download.pytorch.org/whl/cu113



---



### 2. Prediction on yeast data without any fine-tuning: MEDIAR's phase 1, phase 1 fine-tuned, Ensemble Model.

In this section, we perform prediction and evaluation on the whole YeaZ dataset, in order to establish the performance of MEDIAR on the YeaZ data without having seen it.
#### Download (restructured) yeast data

In [ ]:
!gdown [ insert data link ] --fuzzy

In [ ]:
import zipfile
with zipfile.ZipFile("/content/ml-project-2-doughminators/Datasets.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/ml-project-2-doughminators/")

Sanity check: should be 2914 images

In [ ]:
import os
len(os.listdir("/content/ml-project-2-doughminators/Datasets/images"))

2914

#### Inference on whole YeaZ dataset
- switch device accordingly in the config file

Note: this corresponds to the P1FT model in the report.

In [ ]:
!python predict.py --config_path=config/step3_prediction/base_prediction.json


================================================== Configuration ==================================================
{'pred_setups': {'algo_params': {'use_tta': False},
                 'device': 'cuda:0',
                 'exp_name': 'mediar_p1_base',
                 'input_path': 'Datasets/images',
                 'make_submission': True,
                 'model': {'name': 'mediar-former',
                           'params': {'classes': 3,
                                      'decoder_channels': [1024, 512, 256, 128,
                                                           64],
                                      'decoder_pab_channels': 256,
                                      'encoder_name': 'mit_b5',
                                      'in_channels': 3}},
                 'model_path': './weights/from_phase1.pth',
                 'name': 'mediar',
                 'output_path': './results/mediar_base_prediction'}}
Prediction finished: 10_Ref_Zstack_DIC_crop_1_frame_0.

Evaluate results

In [ ]:
!python ./evaluate.py --pred_path=results/mediar_base_prediction --gt_path=Datasets/labels --save_path=evaluation_result

  8% 226/2869 [00:12<02:30, 17.55it/s]/content/ml-project-2-doughminators/./evaluate.py:48: UserWarning: Warning: file 200302age_SW643_cdc13_tetR_06_Z_45_100_crop_1_label.tiff missing
  warnings.warn(f"Warning: file {name} missing")
/content/ml-project-2-doughminators/./evaluate.py:48: UserWarning: Warning: file 200302age_SW643_cdc13_tetR_06_Z_53_100_crop_1_label.tiff missing
  warnings.warn(f"Warning: file {name} missing")
/content/ml-project-2-doughminators/./evaluate.py:48: UserWarning: Warning: file 200302age_SW643_cdc13_tetR_06_Z_65_100_crop_1_label.tiff missing
  warnings.warn(f"Warning: file {name} missing")
/content/ml-project-2-doughminators/./evaluate.py:48: UserWarning: Warning: file 200302age_SW643_cdc13_tetR_08_Z_45_100_crop_1_label.tiff missing
  warnings.warn(f"Warning: file {name} missing")
/content/ml-project-2-doughminators/./evaluate.py:48: UserWarning: Warning: file 200302age_SW643_cdc13_tetR_08_Z_54_100_crop_1_label.tiff missing
  warnings.warn(f"Warning: file {nam

Ensemble model

Note: this corresponds to Ensemble(P1,P1FT)+TTA model in the report.

In [ ]:
!python predict.py --config_path=config/step3_prediction/ensemble_tta.json


================================================== Configuration ==================================================
{'pred_setups': {'algo_params': {'use_tta': True},
                 'device': 'cuda:0',
                 'exp_name': 'mediar_ensemble_tta',
                 'input_path': 'Datasets/images',
                 'make_submission': True,
                 'model': {'name': 'mediar-former',
                           'params': {'classes': 3,
                                      'decoder_channels': [1024, 512, 256, 128,
                                                           64],
                                      'decoder_pab_channels': 256,
                                      'encoder_name': 'mit_b5',
                                      'in_channels': 3}},
                 'model_path1': './weights/from_phase1.pth',
                 'model_path2': './weights/from_phase2.pth',
                 'name': 'ensemble_mediar',
                 'output_path': './results/media

In [ ]:
!python ./evaluate.py --pred_path=results/mediar_ensemble_tta --gt_path=Datasets/labels --save_path=evaluation_result/ensemble_tta

  8% 227/2869 [00:12<01:43, 25.51it/s]/content/ml-project-2-doughminators/./evaluate.py:48: UserWarning: Warning: file 200302age_SW643_cdc13_tetR_06_Z_45_100_crop_1_label.tiff missing
  warnings.warn(f"Warning: file {name} missing")
/content/ml-project-2-doughminators/./evaluate.py:48: UserWarning: Warning: file 200302age_SW643_cdc13_tetR_06_Z_53_100_crop_1_label.tiff missing
  warnings.warn(f"Warning: file {name} missing")
/content/ml-project-2-doughminators/./evaluate.py:48: UserWarning: Warning: file 200302age_SW643_cdc13_tetR_06_Z_65_100_crop_1_label.tiff missing
  warnings.warn(f"Warning: file {name} missing")
/content/ml-project-2-doughminators/./evaluate.py:48: UserWarning: Warning: file 200302age_SW643_cdc13_tetR_08_Z_45_100_crop_1_label.tiff missing
  warnings.warn(f"Warning: file {name} missing")
/content/ml-project-2-doughminators/./evaluate.py:48: UserWarning: Warning: file 200302age_SW643_cdc13_tetR_08_Z_54_100_crop_1_label.tiff missing
  warnings.warn(f"Warning: file {nam

In [ ]:
!zip -r /content/ml-project-2-doughminators/results/mediar_base_prediction.zip /content/ml-project-2-doughminators/results/mediar_base_prediction
!zip -r /content/ml-project-2-doughminators/results/mediar_ensemble_tta.zip /content/ml-project-2-doughminators/results/mediar_ensemble_tta

Inference on the phase 1 model (only pretrained on ImageNet-1k and public datasets).

Note: this corresponds to P1 model in the report.

In [ ]:
!python predict.py --config_path=config/step3_prediction/phase_1_prediction.json
!python ./evaluate.py --pred_path=results/phase_1_prediction --gt_path=Datasets/labels --save_path=evaluation_result/phase_1_prediction
!zip -r /content/ml-project-2-doughminators/results/phase_1_prediction.zip /content/ml-project-2-doughminators/results/phase_1_prediction
from google.colab import files
files.download("/content/ml-project-2-doughminators/results/phase_1_prediction.zip")
files.download("/content/ml-project-2-doughminators/evaluation_result/phase_1_prediction/seg_metric.csv")

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Prediction finished: 3306_pad1_8_Zstack_TL_crop_1_frame_4.tiff; img size = torch.Size([1, 3, 500, 500]); costing: 0.30s
Prediction finished: 3306_pad1_8_Zstack_TL_crop_1_frame_5.tiff; img size = torch.Size([1, 3, 500, 500]); costing: 0.29s
Prediction finished: 3306_pad1_8_Zstack_TL_crop_1_frame_6.tiff; img size = torch.Size([1, 3, 500, 500]); costing: 0.30s
Prediction finished: 3306_pad1_8_Zstack_TL_crop_1_frame_7.tiff; img size = torch.Size([1, 3, 500, 500]); costing: 0.30s
Prediction finished: 3306_pad1_8_Zstack_TL_crop_1_frame_8.tiff; img size = torch.Size([1, 3, 500, 500]); costing: 0.29s
Prediction finished: 3306_pad1_8_Zstack_TL_crop_1_frame_9.tiff; img size = torch.Size([1, 3, 500, 500]); costing: 0.36s
Prediction finished: 3306_pad1_9_Zstack_DIC_crop_1_frame_0.tiff; img size = torch.Size([1, 3, 500, 500]); costing: 0.37s
Prediction finished: 3306_pad1_9_Zstack_DIC_crop_1_frame_1.tiff; img size = torch

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 3. Fine-tuning on YeaZ dataset.

Download the ```path1.pth``` checkpoint

In [ ]:
!gdown https://drive.google.com/file/d/1v5tYYJDqiwTn_mV0KyX5UEonlViSNx4i/view?usp=share_link --fuzzy -O weights/phase1.pth

Downloading...
From: https://drive.google.com/uc?id=1v5tYYJDqiwTn_mV0KyX5UEonlViSNx4i
To: /content/ml-project-2-doughminators/weights/phase1.pth
100% 486M/486M [00:10<00:00, 47.6MB/s]


In [ ]:
!python ./generate_mapping.py --root=./Datasets/


----------- Path Mapping for Labeled Data is Started... -----------

0  mismatches

----------- Path Mapping for Tuning Data is Started... -----------

>>> "official" already exists in path map keys...

----------- Path Mapping for Public Data is Started... -----------

>>> "public" already exists in path map keys...

-------------- Path Mapping is Ended !!! ---------------------------



Check that the mapping was successful, i.e., there are 2914 image/label pairs.

In [ ]:
import pandas as pd
mapping = pd.read_json("train_tools/data_utils/mapping_labeled.json")

In [ ]:
len(mapping["official"])

2914

Fine-tuning the phase 1 model (most general, but without having seen any challenge data).

In [ ]:
!python ./main.py --config_path=config/step2_finetuning/finetuning_yeast.json

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowEllb
  warn(f"Failed to load image Python extension: {e}")

================================================== Configuration ==================================================
{'data_setups': {'labeled': {'amplified': True,
                             'batch_size': 8,
                             'mapping_file': './train_tools/data_utils/mapping_labeled.json',
                             'mapping_file_tuning': './train_tools/data_utils/mapping_labeled.json',
                             'root': './',
                             'valid_portion': 0.2},
                 'public': {'enabled': False,
                            'params': {'batch_size': 1,
                                       'mapping_file': './train_tools/data_utils/mapping_public.json',


In [ ]:
!python ./evaluate.py --pred_path=results/ --gt_path=Datasets/test/labels --save_path=evaluation_result/mediar_FT_YeaZ_test_prediction

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowEllb
  warn(f"Failed to load image Python extension: {e}")
100% 5/5 [00:01<00:00,  2.90it/s]
mean F1 Score: 0.92548 +/- 0.014089487712475566
mean AP Score: 0.86908 +/- 0.023814252539183338


Now we finetune the best model, i.e., "phase 2 fine-tuned". It performs the best on the challenge data, and it also has seen more data than phase 1 alone! (we just change the initial weights in the config file)

However, it may be a bit too specific to challenge data. Let's see.

In [ ]:
!python ./main.py --config_path=config/step2_finetuning/finetuning_yeast.json

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowEllb
  warn(f"Failed to load image Python extension: {e}")

================================================== Configuration ==================================================
{'data_setups': {'labeled': {'amplified': True,
                             'batch_size': 8,
                             'mapping_file': './train_tools/data_utils/mapping_labeled.json',
                             'mapping_file_tuning': './train_tools/data_utils/mapping_labeled.json',
                             'root': './',
                             'valid_portion': 0.2},
                 'public': {'enabled': False,
                            'params': {'batch_size': 1,
                                       'mapping_file': './train_tools/data_utils/mapping_public.json',


Extract zip of the submission (test data inference results) and evaluate it.

In [ ]:
import zipfile
with zipfile.ZipFile("/content/ml-project-2-doughminators/submissions/mediar_ft_challenge_from_phase2_on_YeaZ1219_1535.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/ml-project-2-doughminators/submissions/mediar_ft_challenge_from_phase2_on_YeaZ")

In [ ]:
!python ./evaluate.py --pred_path=submissions/mediar_ft_challenge_from_phase2_on_YeaZ --gt_path=Datasets/test/labels --save_path=evaluation_result/mediar_ft_challenge_from_phase2_on_YeaZ

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowEllb
  warn(f"Failed to load image Python extension: {e}")
  0% 0/6 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "/content/ml-project-2-doughminators/./evaluate.py", line 84, in <module>
    main()
  File "/content/ml-project-2-doughminators/./evaluate.py", line 41, in main
    assert name.endswith(
AssertionError: The suffix of label name .ipynb_checkpoints should be _label.tiff


We find out that this "phase 2 fine-tuned" model, when further fine-tuned on our YeaZ data, continues to learn after 20 epochs. It also outperforms the phase 1 model after fine-tuning on YeaZ. So, let's train the "phase 2 fine-tuned" model further with 20 more epochs. First we download the weights checkpoint.

In [ ]:
!gdown https://drive.google.com/file/d/1p5Ih0_R3u52pX0oOOvuA46FGVgGVVh4i/view?usp=sharing --fuzzy -O weights/ft_from_phase2_on_YeaZ_20_epochs.pth.zip

Downloading...
From: https://drive.google.com/uc?id=1p5Ih0_R3u52pX0oOOvuA46FGVgGVVh4i
To: /content/ml-project-2-doughminators/weights/ft_from_phase2_on_YeaZ_20_epochs.pth.zip
100% 450M/450M [00:19<00:00, 22.7MB/s]


In [ ]:
import zipfile
with zipfile.ZipFile("/content/ml-project-2-doughminators/weights/ft_from_phase2_on_YeaZ_20_epochs.pth.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/ml-project-2-doughminators/weights/ft_from_phase2_on_YeaZ_20_epochs.pth")

Modify the config file with 40 epochs.

In [ ]:
!python ./main.py --config_path=config/step2_finetuning/finetuning_yeast.json

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowEllb
  warn(f"Failed to load image Python extension: {e}")

================================================== Configuration ==================================================
{'data_setups': {'labeled': {'amplified': False,
                             'batch_size': 8,
                             'mapping_file': './train_tools/data_utils/mapping_labeled.json',
                             'mapping_file_tuning': './train_tools/data_utils/mapping_labeled.json',
                             'root': './',
                             'valid_portion': 0.2},
                 'public': {'enabled': False,
                            'params': {'batch_size': 1,
                                       'mapping_file': './train_tools/data_utils/mapping_public.json',

[Error at prediction on test set so we do it manually below]

In [ ]:
!gdown https://drive.google.com/file/d/1yrLncA-wf594qKAE43o3BDP-Za7S1YV0/view?usp=sharing --fuzzy -O weights/

Downloading...
From: https://drive.google.com/uc?id=1yrLncA-wf594qKAE43o3BDP-Za7S1YV0
To: /content/ml-project-2-doughminators/weights/ft_from_phase2_on_YeaZ_40_epochs.pth.zip
100% 450M/450M [00:13<00:00, 32.4MB/s]


In [ ]:
import zipfile
with zipfile.ZipFile("/content/ml-project-2-doughminators/weights/ft_from_phase2_on_YeaZ_40_epochs.pth.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/ml-project-2-doughminators/weights/")

In [ ]:
!python predict.py --config_path=config/step3_prediction/ft_from_phase2_on_YeaZ_40_epochs.json

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowEllb
  warn(f"Failed to load image Python extension: {e}")

================================================== Configuration ==================================================
{'pred_setups': {'algo_params': {'use_tta': False},
                 'device': 'cuda:0',
                 'exp_name': 'mediar_ft_from_phase2_on_YeaZ_40_epochs',
                 'input_path': 'Datasets/test/images',
                 'make_submission': True,
                 'model': {'name': 'mediar-former',
                           'params': {'classes': 3,
                                      'decoder_channels': [1024, 512, 256, 128,
                                                           64],
                                      'decoder_pab_channels': 256,
                

In [ ]:
!python ./evaluate.py --pred_path=results/ft_from_phase2_on_YeaZ_40_epochs --gt_path=Datasets/test/labels --save_path=evaluation_result/ft_from_phase2_on_YeaZ_40_epochs

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowEllb
  warn(f"Failed to load image Python extension: {e}")
100% 5/5 [00:01<00:00,  2.89it/s]
mean F1 Score: 0.9772399999999999 +/- 0.0024343081152557473
mean AP Score: 0.95576 +/- 0.004596530865772574


Note: with test-time augmentation, it slightly reduces metrics to 0.9766 F1 and 0.9545 AP.

In [ ]:
!zip -r /content/ml-project-2-doughminators/results/ft_from_phase2_on_YeaZ_40_epochs.zip /content/ml-project-2-doughminators/results/ft_from_phase2_on_YeaZ_40_epochs

  adding: content/ml-project-2-doughminators/results/ft_from_phase2_on_YeaZ_40_epochs/ (stored 0%)
  adding: content/ml-project-2-doughminators/results/ft_from_phase2_on_YeaZ_40_epochs/YSM3919_pak2Dcdc42mChSW_24h_04_R3D_image_label.tiff (deflated 12%)
  adding: content/ml-project-2-doughminators/results/ft_from_phase2_on_YeaZ_40_epochs/pSP_99_crop_1_im_label.tiff (deflated 24%)
  adding: content/ml-project-2-doughminators/results/ft_from_phase2_on_YeaZ_40_epochs/OD301_01_image_label.tiff (deflated 8%)
  adding: content/ml-project-2-doughminators/results/ft_from_phase2_on_YeaZ_40_epochs/IBC180_004_image_label.tiff (deflated 7%)
  adding: content/ml-project-2-doughminators/results/ft_from_phase2_on_YeaZ_40_epochs/pSP_99_crop_2_im_label.tiff (deflated 18%)


Now we want to perform ensemble prediction with the P1 model.

In [ ]:
!python predict.py --config_path=config/step3_prediction/ensemble_tta_fromphase2_ft_from_phase2_on_YeaZ_40_epochs.json

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowEllb
  warn(f"Failed to load image Python extension: {e}")

================================================== Configuration ==================================================
{'pred_setups': {'algo_params': {'use_tta': True},
                 'device': 'cuda:0',
                 'exp_name': 'mediar_ensemble_tta_fromphase2_ft_fromphase2_on_YeaZ_40_epochs',
                 'input_path': 'Datasets/test/images',
                 'make_submission': True,
                 'model': {'name': 'mediar-former',
                           'params': {'classes': 3,
                                      'decoder_channels': [1024, 512, 256, 128,
                                                           64],
                                      'decoder_pab_channels':

In [ ]:
!python ./evaluate.py --pred_path=results/ensemble_tta_fromphase2_ft_fromphase2_on_YeaZ_40_epochs --gt_path=Datasets/test/labels --save_path=evaluation_result/ensemble_tta_fromphase2_ft_fromphase2_on_YeaZ_40_epochs

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowEllb
  warn(f"Failed to load image Python extension: {e}")
100% 5/5 [00:01<00:00,  2.89it/s]
mean F1 Score: 0.9683999999999999 +/- 0.0036124673008900684
mean AP Score: 0.93932 +/- 0.006762453992449785


doesn't work as well (maybe because the second model used in the ensemble is just not general). We could try it with the P2 model which has also seen challenge data, thus more general. For time constraints we are not able to do so.

Now let's fine-tune further on 40 more epochs. (we modify the config file to restart from the last checkpoint, accumulating the training on the last 40 epochs).

In [ ]:
!python ./main.py --config_path=config/step2_finetuning/finetuning_yeast.json

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowEllb
  warn(f"Failed to load image Python extension: {e}")

================================================== Configuration ==================================================
{'data_setups': {'labeled': {'amplified': False,
                             'batch_size': 8,
                             'mapping_file': './train_tools/data_utils/mapping_labeled.json',
                             'mapping_file_tuning': './train_tools/data_utils/mapping_labeled.json',
                             'root': './',
                             'valid_portion': 0.2},
                 'public': {'enabled': False,
                            'params': {'batch_size': 1,
                                       'mapping_file': './train_tools/data_utils/mapping_public.json',

The process is killed after 8 epochs (48 total) for technical reasons. However we can see that the validation F1 score does not increase anymore (in fact, the best validation F1 score was obtained in an epoch < 40).

### Reproducibility of best results on a test set

Download the weights:

In [ ]:
!gdown [link to weights] --fuzzy -O weights/

Write or re-use a prediction configuration file. In this case we use ```ft_from_phase2_on_YeaZ_80_epochs.json```. Make sure to modify the input path of the test images accordingly, as well as the output path, the device (if the test set is less than 10 images, it should be fine to use a CPU), and the experiment name. It is also possible to activate/deactivate test-time augmentation (TTA) but in our test case it worsens the results.

In [ ]:
!python predict.py --config_path=config/step3_prediction/ft_from_phase2_on_YeaZ_80_epochs.json

Evaluate the predictions:

In [ ]:
!python ./evaluate.py --pred_path=results/ft_fromphase2_on_YeaZ_80_epochs --gt_path=Datasets/test/labels --save_path=evaluation_result/ft_fromphase2_on_YeaZ_80_epochs